In [1]:
## Import All Needed Libraries

import os
import gym
import ray
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.gym.environment import AnimalAIGym
from ray.rllib.agents import ppo

from ray.tune import register_env, tune

from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.models import ModelCatalog
from ray.tune.logger import pretty_print

from cache_model import *
from config import get_cfg
from custom_model import *

Instructions for updating:
non-resource variables are not supported in the long term
2021-04-06 19:33:40,557	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!


In [2]:
## Reuse Wrapper for AnimalAI Environment

class UnityEnvWrapper(gym.Env):
    def __init__(self, env_config):
        self.vector_index = env_config.vector_index
        self.worker_index = env_config.worker_index
        self.worker_id = env_config["unity_worker_id"] + env_config.worker_index
        self.env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = self.worker_id,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig(env_config['arena_to_train'])
        )
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
        
    def reset(self):
        return self.env.reset()
    
    def step(self, action):
        return self.env.step(action)

In [3]:
## Setup configuration to use

conf = {
    "num_workers": 0,
    "env_config": {
        "unity_worker_id": 67,
        "arena_to_train": '../examples/configurations/curriculum/5.yml'
        },
        "model": {
            "custom_model": 'my_cnn_rnn_model',
            "custom_model_config": {},
        },
        "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "2")),
        "num_workers": 1,  # parallelism
        "framework": "torch",
        "train_batch_size": 500,
        
       }
conf

{'num_workers': 1,
 'env_config': {'unity_worker_id': 67,
  'arena_to_train': '../examples/configurations/curriculum/5.yml'},
 'model': {'custom_model': 'my_cnn_rnn_model', 'custom_model_config': {}},
 'num_gpus': 2,
 'framework': 'torch',
 'train_batch_size': 500}

In [4]:
## Setup and register environment
ray.shutdown()
ray.init(num_cpus=4, num_gpus=2)

# Register custom models so that we can give the ID to the policy trainer
# ModelCatalog.register_custom_model("my_fc_model", MyFCForwardModel)
ModelCatalog.register_custom_model("my_rnn_model", MyRNNModel)
ModelCatalog.register_custom_model("my_convgru_model", MyConvGRUModel)  # NOTE: Only works with image observations.
ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)

register_env("unity_env", lambda config: UnityEnvWrapper(config))

2021-04-06 19:35:22,976	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266


In [5]:
trainer = PPOTrainer(config=conf, env= "unity_env")

2021-04-06 19:35:27,631	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=18850) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=18850) Instructions for updating:
(pid=18850) non-resource variables are not supported in the long term


(pid=18850) Found path: /home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI.x86_64
(pid=18850) Mono path[0] = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/Managed'
(pid=18850) Mono config path = '/home/azibit/COMS_673/AnimalAI-Olympics/cachey/../examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=18850) Preloaded 'lib_burst_generated.so'
(pid=18850) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=18850) Preloaded 'ScreenSelector.so'
(pid=18850) Display 0 'C32F391 32"': 1920x1080 (primary device).
(pid=18850) Logging to /home/azibit/.config/unity3d/Unity Technologies/UnityEnvironment/Player.log


(pid=18850) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=18850) INFO:mlagents_envs:Connected new brain:
(pid=18850) AnimalAI?team=0
(pid=18850) INFO:gym_unity:1 agents within environment.
(pid=18850) 2021-04-06 19:35:33,018	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=18850) 2021-04-06 19:35:33,056	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!
2021-04-06 19:35:33,176	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-04-06 19:35:35,251	WARNING util.py:47 -- Install gputil for GPU system monitoring.


In [6]:
## Train the model
checkpoint = ""
for i in range(1002):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(pretty_print(result))

   if i % 200 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)

(pid=18850) 2021-04-06 19:36:09,868	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
2021-04-06 19:36:14,299	WARNING sgd.py:69 -- Not time-shuffling RNN data for SGD.


custom_metrics: {}
date: 2021-04-06_19-36-16
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 2
experiment_id: a88c6111ff18408aad4cfbb03e696fbe
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 0.2
      cur_lr: 5.0e-05
      entropy: 2.137380027770996
      entropy_coeff: 0.0
      kl: 0.025680503621697427
      policy_loss: -0.08258256614208222
      total_loss: -0.07367224097251893
      vf_explained_var: 0.6808925271034241
      vf_loss: 0.003774236887693405
  num_steps_sampled: 600
  num_steps_trained: 600
iterations_since_restore: 1
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 11.401666666666669
  ram_util_percent: 43.2
pid: 18605
policy_reward_max: {}
policy_reward_mean: {}
policy_reward_min: {}
sampler_perf:
  mean

custom_metrics: {}
date: 2021-04-06_19-36-54
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 16
experiment_id: a88c6111ff18408aad4cfbb03e696fbe
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 1.0125000000000002
      cur_lr: 5.0e-05
      entropy: 2.098140776157379
      entropy_coeff: 0.0
      kl: 0.014817873015999794
      policy_loss: -0.35353048518300056
      total_loss: -0.3361012227833271
      vf_explained_var: 0.7000163793563843
      vf_loss: 0.002426172693958506
  num_steps_sampled: 4200
  num_steps_trained: 4200
iterations_since_restore: 7
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 19.31111111111111
  ram_util_percent: 43.7
pid: 18605
policy_reward_max: {}
policy_reward_mean: {}
policy_reward_min: {}
samp

custom_metrics: {}
date: 2021-04-06_19-37-32
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 3
episodes_total: 31
experiment_id: a88c6111ff18408aad4cfbb03e696fbe
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 1.5187500000000003
      cur_lr: 5.0e-05
      entropy: 2.0195152282714846
      entropy_coeff: 0.0
      kl: 0.01767448764294386
      policy_loss: -0.04557156562805176
      total_loss: -0.016058385372161865
      vf_explained_var: 0.6289110779762268
      vf_loss: 0.002670068759471178
  num_steps_sampled: 7800
  num_steps_trained: 7800
iterations_since_restore: 13
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 18.288888888888888
  ram_util_percent: 43.51111111111111
pid: 18605
policy_reward_max: {}
policy_reward_mean: {}
policy_re

custom_metrics: {}
date: 2021-04-06_19-38-10
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 45
experiment_id: a88c6111ff18408aad4cfbb03e696fbe
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 1.5187500000000003
      cur_lr: 5.0e-05
      entropy: 2.0090863704681396
      entropy_coeff: 0.0
      kl: 0.015280828066170215
      policy_loss: -0.10137143731117249
      total_loss: -0.07410805821418762
      vf_explained_var: 0.7234721779823303
      vf_loss: 0.004055599588900805
  num_steps_sampled: 11400
  num_steps_trained: 11400
iterations_since_restore: 19
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 18.988888888888887
  ram_util_percent: 43.53333333333333
pid: 18605
policy_reward_max: {}
policy_reward_mean: {}
policy_

...

custom_metrics: {}
date: 2021-04-06_21-20-57
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 2335
experiment_id: a88c6111ff18408aad4cfbb03e696fbe
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 3.4171875000000003
      cur_lr: 5.0e-05
      entropy: 1.0305641889572144
      entropy_coeff: 0.0
      kl: 0.009988307603634894
      policy_loss: -0.23704037815332413
      total_loss: -0.20021551847457886
      vf_explained_var: 0.7355868816375732
      vf_loss: 0.0026929265295621008
  num_steps_sampled: 583800
  num_steps_trained: 583800
iterations_since_restore: 973
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 38.17
  ram_util_percent: 57.120000000000005
pid: 18605
policy_reward_max: {}
policy_reward_mean: {}
policy_reward

custom_metrics: {}
date: 2021-04-06_21-21-39
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 3
episodes_total: 2350
experiment_id: a88c6111ff18408aad4cfbb03e696fbe
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 3.4171875000000003
      cur_lr: 5.0e-05
      entropy: 1.033822476863861
      entropy_coeff: 0.0
      kl: 0.011285018175840378
      policy_loss: -0.1602741777896881
      total_loss: -0.11915672197937965
      vf_explained_var: 0.7852039337158203
      vf_loss: 0.0025544499803800136
  num_steps_sampled: 587400
  num_steps_trained: 587400
iterations_since_restore: 979
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 30.35
  ram_util_percent: 57.279999999999994
pid: 18605
policy_reward_max: {}
policy_reward_mean: {}
policy_reward_m

custom_metrics: {}
date: 2021-04-06_21-22-21
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 2364
experiment_id: a88c6111ff18408aad4cfbb03e696fbe
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 3.4171875000000003
      cur_lr: 5.0e-05
      entropy: 1.0565763115882874
      entropy_coeff: 0.0
      kl: 0.014111665077507496
      policy_loss: -0.061953261494636536
      total_loss: -0.010068334639072418
      vf_explained_var: 0.730987548828125
      vf_loss: 0.0036627230001613498
  num_steps_sampled: 591000
  num_steps_trained: 591000
iterations_since_restore: 985
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 34.76
  ram_util_percent: 57.06
pid: 18605
policy_reward_max: {}
policy_reward_mean: {}
policy_reward_min: {}
sam

custom_metrics: {}
date: 2021-04-06_21-23-03
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 3
episodes_total: 2379
experiment_id: a88c6111ff18408aad4cfbb03e696fbe
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 3.4171875000000003
      cur_lr: 5.0e-05
      entropy: 0.9793567359447479
      entropy_coeff: 0.0
      kl: 0.01008391275536269
      policy_loss: -0.046229854226112366
      total_loss: -0.009114093147218227
      vf_explained_var: 0.8027640581130981
      vf_loss: 0.0026571328344289213
  num_steps_sampled: 594600
  num_steps_trained: 594600
iterations_since_restore: 991
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 37.17999999999999
  ram_util_percent: 57.089999999999996
pid: 18605
policy_reward_max: {}
policy_reward_mean: {}


custom_metrics: {}
date: 2021-04-06_21-23-45
done: false
episode_len_mean: 250.0
episode_reward_max: -0.9999999310821295
episode_reward_mean: -0.9999999310821295
episode_reward_min: -0.9999999310821295
episodes_this_iter: 2
episodes_total: 2393
experiment_id: a88c6111ff18408aad4cfbb03e696fbe
hostname: azibit-Lenovo-Y520-15IKBM
info:
  learner:
    default_policy:
      allreduce_latency: 0.0
      cur_kl_coeff: 3.4171875000000003
      cur_lr: 5.0e-05
      entropy: 0.9850463718175888
      entropy_coeff: 0.0
      kl: 0.012543221702799201
      policy_loss: -0.0697830319404602
      total_loss: -0.02242763340473175
      vf_explained_var: 0.7762529850006104
      vf_loss: 0.004492853186093271
  num_steps_sampled: 598200
  num_steps_trained: 598200
iterations_since_restore: 997
node_ip: 192.168.1.4
num_healthy_workers: 1
off_policy_estimator: {}
perf:
  cpu_util_percent: 36.51
  ram_util_percent: 57.17
pid: 18605
policy_reward_max: {}
policy_reward_mean: {}
policy_reward_min: {}
sample

In [23]:
trainer

In [10]:
## Trying to test the trained agent on an environment

env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = 150,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig('../examples/configurations/curriculum/0.yml')
)

INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
INFO:mlagents_envs:Connected new brain:
AnimalAI?team=0
INFO:gym_unity:1 agents within environment.


In [11]:
# run for number of steps
episode_reward = 0
done = False
recurrent_initial_state = trainer.get_policy().model.get_initial_state()
obs = env.reset()


# print(obs.shape)
while not done:
    action = trainer.compute_action(obs, recurrent_initial_state)
    action, recurrent_initial_state = action[0], action[1]
    print("ACTION: ", action)
#     action, recurrent_initial_state = action[0], action[1]
    obs, reward, done, info = env.step(action)
    
    print("REWARD: ", reward)
#     print(info)
    episode_reward += reward

ACTION:  5
REWARD:  -0.0039999997
ACTION:  7
REWARD:  -0.0039999997
ACTION:  8
REWARD:  -0.0039999997
ACTION:  5
REWARD:  -0.0039999997
ACTION:  4
REWARD:  -0.0039999997
ACTION:  4
REWARD:  -0.0039999997
ACTION:  1
REWARD:  -0.0039999997
ACTION:  2
REWARD:  -0.0039999997
ACTION:  0
REWARD:  -0.0039999997
ACTION:  0
REWARD:  -0.0039999997
ACTION:  7
REWARD:  -0.0039999997
ACTION:  0
REWARD:  -0.0039999997
ACTION:  8
REWARD:  -0.0039999997
ACTION:  8
REWARD:  -0.0039999997
ACTION:  4
REWARD:  -0.0039999997
ACTION:  1
REWARD:  -0.0039999997
ACTION:  4
REWARD:  -0.0039999997
ACTION:  8
REWARD:  -0.0039999997
ACTION:  0
REWARD:  -0.0039999997
ACTION:  8
REWARD:  -0.0039999997
ACTION:  8
REWARD:  -0.0039999997
ACTION:  8
REWARD:  -0.0039999997
ACTION:  6
REWARD:  -0.0039999997
ACTION:  6
REWARD:  -0.0039999997
ACTION:  0
REWARD:  -0.0039999997
ACTION:  0
REWARD:  -0.0039999997
ACTION:  3
REWARD:  -0.0039999997
ACTION:  2
REWARD:  -0.0039999997
ACTION:  6
REWARD:  -0.0039999997
ACTION:  3
REW

In [12]:
episode_reward

-0.9999999310821295

In [13]:
env.close()

INFO:mlagents_envs:Environment shut down with return code 0.
